# ImgAE-Dx HuggingFace Training

This notebook implements T4-optimized training for medical image anomaly detection using HuggingFace datasets.

## Quick Start for HuggingFace Training:
1. Run cells in order, SKIP cell that loads Kaggle/Synthetic data
2. Ensure CONFIG has 'dataset_source': 'huggingface'

## 1. Setup Colab Environment

In [ ]:
# Check GPU and mount Google Drive
import torch
import os

# Check GPU
if torch.cuda.is_available():
    gpu_info = !nvidia-smi --query-gpu=name,memory.total --format=csv,noheader
    print(f"GPU: {gpu_info[0]}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"PyTorch Version: {torch.__version__}")
else:
    print("⚠️ No GPU detected! Please enable GPU in Runtime > Change runtime type")

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("✅ Google Drive mounted")

## 2. Install ImgAE-Dx

In [ ]:
# Clone repository if not exists
if not os.path.exists('/content/ImgAE-Dx'):
    !git clone https://github.com/kinhluan/ImgAE-Dx.git /content/ImgAE-Dx
    %cd /content/ImgAE-Dx
else:
    %cd /content/ImgAE-Dx
    !git pull

# Add the src directory to Python path
import sys
if '/content/ImgAE-Dx/src' not in sys.path:
    sys.path.append('/content/ImgAE-Dx/src')

# Install dependencies
!pip install -e .
!pip install datasets transformers accelerate
!pip install wandb --upgrade

print("✅ ImgAE-Dx installed")

## 3. Configuration

In [ ]:
# T4-optimized configuration
CONFIG = {
    # Model settings
    'model_type': 'unet',  # 'unet', 'reversed_ae', or 'both'
    
    # Dataset source
    'dataset_source': 'huggingface',  # 'huggingface', 'kaggle', or 'synthetic'
    
    # HuggingFace Dataset settings
    'hf_dataset': 'hf-vision/chest-xray-pneumonia',  # Working chest X-ray dataset
    'hf_config': None,  # Dataset configuration (None for default)
    'hf_split': 'train',
    'hf_streaming': False,  # Set to False for this dataset
    'hf_token': None,  # Add your HuggingFace token here if using private datasets
    'image_column': 'image',  # Column name containing images
    'label_column': 'labels',  # Column name containing labels
    
    # Training settings (T4-optimized)
    'samples': 3000,
    'epochs': 20,
    'batch_size': 48,  # T4-optimized with mixed precision
    'learning_rate': 1e-4,
    'image_size': 128,
    
    # T4 optimizations
    'mixed_precision': True,
    'memory_limit': 14,  # GB (leave 2GB for system)
    'gradient_accumulation_steps': 1,
    'num_workers': 4,
    
    # Checkpointing
    'checkpoint_dir': '/content/drive/MyDrive/imgae_dx_checkpoints',
    'save_frequency': 5,  # Save every N epochs
    'resume_from_checkpoint': None,  # Path to checkpoint to resume from
    
    # Logging
    'use_wandb': False,  # Set to True to use W&B
    'wandb_project': 'imgae-dx-t4-colab',
    'wandb_run_name': None,  # Auto-generated if None
}

# Create directories
os.makedirs(CONFIG['checkpoint_dir'], exist_ok=True)
os.makedirs('/content/outputs/logs', exist_ok=True)
os.makedirs('/content/outputs/checkpoints', exist_ok=True)

print("Configuration set!")
print(f"Model: {CONFIG['model_type']}")
print(f"Dataset source: {CONFIG['dataset_source']}")
print(f"HuggingFace Dataset: {CONFIG['hf_dataset']}")
print(f"Batch size: {CONFIG['batch_size']} (T4-optimized)")
print(f"Mixed precision: {CONFIG['mixed_precision']}")

## 4. Setup Weights & Biases (Optional)

In [ ]:
if CONFIG['use_wandb']:
    import wandb
    
    # Login to W&B (you'll need to paste your API key)
    wandb.login()
    
    # Initialize run
    run_name = CONFIG['wandb_run_name'] or f"{CONFIG['model_type']}_t4_{CONFIG['samples']}samples"
    wandb.init(
        project=CONFIG['wandb_project'],
        name=run_name,
        config=CONFIG
    )
    print(f"✅ W&B initialized: {run_name}")
else:
    print("W&B logging disabled")

## 5. Load HuggingFace Dataset

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

# Load dataset
print(f"Loading dataset: {CONFIG['hf_dataset']}...")

try:
    # Authentication if needed
    auth_kwargs = {'use_auth_token': CONFIG['hf_token']} if CONFIG['hf_token'] else {}
    
    # Load dataset
    dataset = load_dataset(
        CONFIG['hf_dataset'],
        CONFIG['hf_config'],
        split=CONFIG['hf_split'],
        **auth_kwargs
    )
    
    # Take only the specified number of samples
    if len(dataset) > CONFIG['samples']:
        dataset = dataset.select(range(CONFIG['samples']))
    
    print(f"✅ Dataset loaded")
    print(f"Dataset features: {dataset.features}")
    print(f"Number of samples: {len(dataset)}")
    
except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    print("\n📋 Alternative chest X-ray datasets you can try:")
    print("1. 'hf-vision/chest-xray-pneumonia' (default)")
    print("2. 'Francesco/chest-xray-pneumonia'")
    print("3. 'Falah/Chest-X-Ray-Images-Pneumonia'")
    raise

# Define transforms
transform = transforms.Compose([
    transforms.Resize((CONFIG['image_size'], CONFIG['image_size'])),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Custom dataset wrapper
class HFImageDataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, transform):
        self.dataset = hf_dataset
        self.transform = transform
        
    def __len__(self):
        return len(self.dataset)
        
    def __getitem__(self, idx):
        item = self.dataset[idx]
        
        # Get image from the dataset - try different column names
        image = None
        for col in [CONFIG['image_column'], 'image', 'img', 'pixel_values']:
            if col in item:
                image = item[col]
                break
        
        if image is None:
            raise ValueError(f"No image column found. Available columns: {list(item.keys())}")
        
        # Convert to PIL if needed
        if not isinstance(image, Image.Image):
            if isinstance(image, np.ndarray):
                image = Image.fromarray(image)
            else:
                image = Image.fromarray(np.array(image))
        
        # Ensure RGB format for grayscale conversion
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        # Apply transforms
        image_tensor = self.transform(image)
        
        return image_tensor

# Create dataset instance
train_dataset = HFImageDataset(dataset, transform)

# Create dataloader
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=True,
    num_workers=CONFIG['num_workers'],
    pin_memory=True,
    drop_last=True  # Drop last incomplete batch
)

print(f"✅ DataLoader created with {len(train_dataset)} images")
print(f"Batch size: {CONFIG['batch_size']}")

## 6. Initialize Models

In [ ]:
from imgae_dx.models import UNet, ReversedAutoencoder
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model initialization based on config
models_to_train = []

if CONFIG['model_type'] in ['unet', 'both']:
    unet = UNet(
        in_channels=1,
        out_channels=1,
        features=[64, 128, 256, 512]
    ).to(device)
    models_to_train.append(('unet', unet))
    print("✅ U-Net initialized")

if CONFIG['model_type'] in ['reversed_ae', 'both']:
    reversed_ae = ReversedAutoencoder(
        in_channels=1,
        latent_dim=128,
        image_size=CONFIG['image_size']
    ).to(device)
    models_to_train.append(('reversed_ae', reversed_ae))
    print("✅ Reversed Autoencoder initialized")

# Count parameters
for name, model in models_to_train:
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"{name}: {trainable_params:,} trainable parameters")

## 7. T4-Optimized Training

In [ ]:
import time
from torch.cuda.amp import GradScaler, autocast
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# T4 optimizations
if torch.cuda.is_available():
    # Enable cuDNN benchmarking for T4
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    
    # Set memory fraction
    torch.cuda.set_per_process_memory_fraction(CONFIG['memory_limit'] / 16.0)
    
    print(f"✅ T4 optimizations enabled")
    print(f"Memory limit: {CONFIG['memory_limit']}GB")

def train_model(model_name, model, train_loader, config):
    """T4-optimized training function"""
    print(f"\n🚀 Training {model_name}...")
    
    # Setup optimizer and loss
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    criterion = nn.MSELoss()
    
    # Mixed precision setup
    scaler = GradScaler() if config['mixed_precision'] else None
    
    # Training history
    history = {'train_loss': []}
    best_loss = float('inf')
    
    # Training loop
    for epoch in range(config['epochs']):
        model.train()
        epoch_loss = 0
        batch_count = 0
        
        # Progress bar
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']}")
        
        for batch_idx, images in enumerate(pbar):
            images = images.to(device)
            
            # Mixed precision training
            if config['mixed_precision']:
                with autocast():
                    reconstructed = model(images)
                    loss = criterion(reconstructed, images)
                
                # Backward pass with scaler
                optimizer.zero_grad()
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                # Standard training
                reconstructed = model(images)
                loss = criterion(reconstructed, images)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            # Update metrics
            epoch_loss += loss.item()
            batch_count += 1
            
            # Update progress bar
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
            
            # Log to W&B
            if config['use_wandb'] and batch_idx % 10 == 0:
                wandb.log({
                    f'{model_name}_batch_loss': loss.item(),
                    'epoch': epoch,
                    'batch': batch_idx
                })
        
        # Calculate epoch metrics
        avg_loss = epoch_loss / batch_count
        history['train_loss'].append(avg_loss)
        
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")
        
        # Log epoch metrics
        if config['use_wandb']:
            wandb.log({
                f'{model_name}_epoch_loss': avg_loss,
                'epoch': epoch + 1
            })
        
        # Save checkpoint
        if (epoch + 1) % config['save_frequency'] == 0 or avg_loss < best_loss:
            checkpoint_path = f"{config['checkpoint_dir']}/{model_name}_epoch_{epoch+1}.pth"
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
                'config': config
            }, checkpoint_path)
            
            if avg_loss < best_loss:
                best_loss = avg_loss
                # Save best model
                best_path = f"{config['checkpoint_dir']}/{model_name}_best.pth"
                torch.save(model.state_dict(), best_path)
                print(f"✅ Best model saved: {best_path}")
    
    return model, history

# Train models
all_histories = {}
trained_models = {}

for model_name, model in models_to_train:
    start_time = time.time()
    
    # Train model
    trained_model, history = train_model(model_name, model, train_loader, CONFIG)
    
    # Store results
    trained_models[model_name] = trained_model
    all_histories[model_name] = history
    
    # Calculate training time
    training_time = (time.time() - start_time) / 60
    print(f"\n✅ {model_name} training completed in {training_time:.1f} minutes")
    
    # Clear cache between models
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

## 8. Visualize Training Results

In [ ]:
# Plot training curves
plt.figure(figsize=(10, 6))

for model_name, history in all_histories.items():
    plt.plot(history['train_loss'], label=f'{model_name} train loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Comparison')
plt.legend()
plt.grid(True)
plt.show()

# Save plot
plt.savefig(f'{CONFIG["checkpoint_dir"]}/training_curves.png', dpi=300, bbox_inches='tight')
print("✅ Training curves saved")

## 9. Test Reconstruction Quality

In [ ]:
# Test reconstruction on a few samples
def visualize_reconstructions(models, data_loader, num_samples=5):
    """Visualize original and reconstructed images"""
    
    # Get sample batch
    sample_batch = next(iter(data_loader))[:num_samples].to(device)
    
    fig, axes = plt.subplots(len(models) + 1, num_samples, figsize=(15, 3 * (len(models) + 1)))
    
    # Original images
    for i in range(num_samples):
        img = sample_batch[i].cpu().squeeze().numpy()
        axes[0, i].imshow(img, cmap='gray')
        axes[0, i].axis('off')
        if i == 0:
            axes[0, i].set_ylabel('Original', fontsize=12)
    
    # Reconstructions
    for idx, (model_name, model) in enumerate(models.items()):
        model.eval()
        with torch.no_grad():
            recon = model(sample_batch)
        
        for i in range(num_samples):
            img = recon[i].cpu().squeeze().numpy()
            axes[idx + 1, i].imshow(img, cmap='gray')
            axes[idx + 1, i].axis('off')
            if i == 0:
                axes[idx + 1, i].set_ylabel(model_name, fontsize=12)
    
    plt.suptitle('Reconstruction Comparison', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Save figure
    plt.savefig(f'{CONFIG["checkpoint_dir"]}/reconstruction_comparison.png', dpi=300, bbox_inches='tight')
    print("✅ Reconstruction comparison saved")

# Visualize reconstructions
if trained_models:
    visualize_reconstructions(trained_models, train_loader)

## 10. Save Final Results

In [ ]:
import json
from datetime import datetime

# Prepare summary
summary = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'config': CONFIG,
    'results': {}
}

for model_name, history in all_histories.items():
    summary['results'][model_name] = {
        'final_loss': history['train_loss'][-1],
        'best_loss': min(history['train_loss']),
        'epochs_trained': len(history['train_loss'])
    }

# Save summary
summary_path = f"{CONFIG['checkpoint_dir']}/training_summary.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print("\n🎯 Training Summary:")
print("=" * 50)
for model_name, results in summary['results'].items():
    print(f"\n{model_name.upper()}:")
    print(f"  Final Loss: {results['final_loss']:.4f}")
    print(f"  Best Loss: {results['best_loss']:.4f}")
    print(f"  Epochs: {results['epochs_trained']}")

print(f"\n✅ Summary saved to: {summary_path}")
print(f"✅ Checkpoints saved to: {CONFIG['checkpoint_dir']}")

# Finish W&B run
if CONFIG['use_wandb']:
    wandb.finish()
    print("✅ W&B run finished")